In [1]:
from image_ops import Dataset_Operations
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from clustering import k_means
data = np.load("baseline.npy")

In [ ]:
dataset = Dataset_Operations(data, metric='l2')
clustering = k_means(60, 100)
clustering.train(dataset, niter=3, init='k++')

In [ ]:
labels = clustering.labels
centers = clustering.centers
counts = dict()
for label in labels:
    counts[label] = counts[label] + 1 if label in counts else 1
counts = [(label, counts[label]) for label in counts]
center_counts = [(centers[int(label[0])], label[1]) for label in counts]
sorted_centers = sorted(center_counts, key=lambda x:x[1], reverse=True)
sorted_centers = [c[0] for c in sorted_centers]
fig, ax = plt.subplots(figsize=(30, 30))
ax.imshow(montage(np.array(sorted_centers)), vmin=-2, vmax=3)
plt.show()
counts = sorted(counts, key=lambda x:x[1], reverse=True)
sorted_labels = np.zeros(len(labels))
for i, entry in enumerate(counts):
    sorted_labels[labels == entry[0]] = i
labels = sorted_labels

plt.hist(labels, bins=40)
plt.show()

In [2]:
from tqdm import tqdm
n_angles = 100
angles = [360/n_angles * i for i in range(n_angles)]
im = data[0]
for i in tqdm(range(10000)):
    for j in angles:
        ndimage.rotate(im, j, reshape=False)
    

  0%|          | 41/10000 [00:08<36:03,  4.60it/s]


KeyboardInterrupt: 

In [ ]:
from skimage.util import montage

In [ ]:
dataset2 = Dataset_Operations(data, metric='wemd')
clustering2 = k_means(60, 100)
clustering2.train(dataset2,niter=5, init='k++')

In [3]:
clustering2.view_centers()

NameError: name 'clustering2' is not defined

In [ ]:
rotations = np.load("angles.npy")

def angle(R1, R2, mod=False):
    dot = R1.T[:,2].T @ R2.T[:,2]
    if np.abs(dot) > 1:
        dot = dot/ np.abs(dot)
    theta1 = np.arccos(dot)* (180/np.pi)
    dot = (-1*R1.T[:,2].T) @ R2.T[:,2]
    if np.abs(dot) > 1:
        dot = dot/ np.abs(dot)
    theta2 = np.arccos(dot)* (180/np.pi)
    if mod:
        return min(theta1, theta2)
    return theta1

def cluster_histogram(i, rotations, labels, mod=False):
    rots = rotations[labels == i]
    angles = []
    for i in range(len(rots)):
        for j in range(i+1, len(rots)):
            angles.append(angle(rots[i], rots[j], mod))
    return angles

def plot_histograms(hists):
    fig = plt.figure(figsize=(30,30))
    n = len(hists)
    cols = 7
    rows = np.ceil(n/7)
    for i, hist in enumerate(hists):
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.hist(hist, bins=180)
    plt.show()
    
    
def ordered_plot(labels, histograms):
    counts = dict()
    for label in labels:
        counts[label] = counts[label] + 1 if label in counts else 1
    counts = [(label, counts[label]) for label in counts]
    hist_counts = [(histograms[int(label[0])], label[1]) for label in counts]
    sorted_hists = sorted(hist_counts, key=lambda x:x[1], reverse=True)
    sorted_hists = [h[0] for h in sorted_hists]
    plot_histograms(sorted_hists)
        


emd_metrics = [cluster_histogram(i, rotations, clustering2.labels) for i in range(60)]

ordered_plot(clustering2.labels, emd_metrics)

